In [3]:
import numpy as np
import pandas as pd
import cv2 as cv
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from nltk import FreqDist
from nltk.stem import PorterStemmer

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from keras.preprocessing import text, sequence

In [2]:
df=pd.read_csv('mon_data.csv')
df.head()

,filename,url,description,type,filepath,scraped,old_filename,old_filepath,text_image
0,img_0000000.jpg,https://templates.invoicehome.com/modele-factu...,100 modèles gratuits de factures | Téléchargem...,0,./final/img_0000000.jpg,1,img_0000000.jpg,./facture/img_0000000.jpg,FACTURE\n\nLOGO\n\nJoanna Binet\n48 Coubertin\...
1,img_0000001.jpg,https://templates.invoicehome.com/modele-factu...,100 modèles gratuits de factures | Téléchargem...,0,./final/img_0000001.jpg,1,img_0000001.jpg,./facture/img_0000001.jpg,Joanna Binet\n48 Coubertin\n31400 Paris\n\nFAC...
2,img_0000002.jpg,https://www.coover.fr/wp-content/uploads/2021/...,Facture proforma : modèle à télécharger gratui...,0,./final/img_0000002.jpg,1,img_0000002.jpg,./facture/img_0000002.jpg,FACTURE\n\nMon entreprise : Nom de la société\...
3,img_0000003.jpg,https://templates.invoicehome.com/modele-factu...,100 modèles gratuits de factures | Téléchargem...,0,./final/img_0000003.jpg,1,img_0000003.jpg,./facture/img_0000003.jpg,Joanna Binet\n48 Coubertin\n31400 Paris\n\nFAC...
4,img_0000004.jpg,https://files.intum.com/v/vosfactures/files/31...,Base de Connaissances - Vos Factures - Créer u...,0,./final/img_0000004.jpg,1,img_0000004.jpg,./facture/img_0000004.jpg,Payer en ligne >\nFACTURE No\n\nSFIDELI\n\nF/0...


In [23]:
data=df.iloc[:,[-1,3]]

In [29]:
from gensim.models import Word2Vec
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     #vector_size=100,
                     alpha=0.03, 
                     negative=10,
                     seed=1)

In [30]:
from nltk.corpus import stopwords
from nltk.tokenize.regexp import  RegexpTokenizer
stop_words = set(stopwords.words('french'))
stop_words.update(["." , ",",':'])
tokenizer=RegexpTokenizer(("[a-zA-Zé]{4,}"))
def stop_words_filtering(liste):
    liste_new=[]
    for i in liste:
        if i not in stop_words:
            liste_new.append(i)
        else:
            continue 
    return liste_new
sentences=[]
for txt in data.text_image:
    mots=stop_words_filtering(tokenizer.tokenize(str(txt).lower()))
    sentences.append(mots)

In [44]:
sentences[478]
w2v_model.build_vocab(sentences, progress_per=10000)
words = list(w2v_model.wv.index_to_key)
print('Vocabulary size: %d' % len(words))
print(words)

Vocabulary size: 1031
['with', 'that', 'date', 'from', 'total', 'this', 'tobacco', 'will', 'have', 'research', 'source', 'other', 'montant', 'university', 'salaire', 'were', 'cotisations', 'which', 'your', 'base', 'taux', 'smoking', 'valeur', 'cigarette', 'revenu', 'sociale', 'year', 'code', 'name', 'paie', 'health', 'these', 'facture', 'more', 'cigarettes', 'heures', 'number', 'bulletin', 'been', 'employeur', 'they', 'service', 'paris', 'than', 'york', 'information', 'their', 'smoke', 'state', 'payer', 'brut', 'company', 'carte', 'american', 'page', 'imposable', 'study', 'only', 'sécurité', 'about', 'time', 'program', 'industrydocuments', 'https', 'brand', 'adresse', 'paiement', 'also', 'report', 'data', 'contributions', 'national', 'years', 'there', 'part', 'assurance', 'each', 'medical', 'medicine', 'product', 'public', 'would', 'such', 'retraite', 'smokers', 'department', 'should', 'please', 'when', 'high', 'amount', 'cancer', 'subject', 'form', 'address', 'some', 'description', 'c

In [37]:
w2v_model.train(sentences,total_examples=w2v_model.corpus_count, epochs=30)

(1569424, 3774840)

In [43]:
mot_vu=w2v_model.wv['facture']

print('Shape of one vector: ', mot_vu.shape, '\n')
print(mot_vu )


Shape of one vector:  (100,) 

[ 0.7982277   0.23931201 -0.73301387 -1.3355633   0.87679255 -0.2088257
  0.62555754  0.08889236 -1.5251461   0.29967672 -0.26125118  0.89874536
 -0.5805324  -0.21915938  1.1480118  -0.3615557  -0.9641476   0.43201298
 -0.07434788 -0.36297783  1.0155146   0.00912657  0.8343874   0.8010467
 -1.48913    -1.8934053   0.82238656  0.4892439  -0.56246144 -1.2215469
 -0.24354894 -0.5810873   0.10998055 -1.418808    0.66060305 -1.308946
 -1.1090803   0.00211307 -0.4525328   0.28499308  0.09512299 -0.14459167
 -0.6034677   0.52758455 -0.5166448   1.4796349  -0.64522237 -1.017558
  0.7121931   0.51336074 -0.30659354 -0.14787123 -0.2544703   0.96566206
 -1.1318887   0.25531533 -0.3330239   0.7814866   1.7546153  -0.3849063
  0.67525125 -1.1422726  -1.3770951  -0.27337214 -0.7685881   0.7825532
 -0.58689106  0.6300455  -0.1367124   0.53386724 -0.30929708  0.48221835
  0.16113286 -1.0098851   1.082639    1.3024272  -0.61814386 -0.10794216
  0.29132313  0.09849928  0.2